In [ ]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import re

# 데이터 불러오기
df = pd.read_csv("전라남도_병원_의원현황.csv", encoding="cp949")

# 주소 컬럼 만들기
df["주소"] = df["소재지(도로명)"].fillna(df["소재지(지번)"])

def clean_address_v2(addr):
    if pd.isna(addr): return ""
    
    # 1. 괄호, 콤마 제거
    addr = re.sub(r"\(.*?\)", "", addr)
    addr = re.sub(r",.*", "", addr)
    
    # 2. '번지', '호', '층' 제거
    addr = re.sub(r"\d+번지", "", addr)
    addr = re.sub(r"\d+호", "", addr)
    addr = re.sub(r"\d+층", "", addr)

    # 3. '리'로 끝나는 지명 + 숫자 제거
    addr = re.sub(r"리\s*\d+(-\d+)?", "리", addr)
    
    # 4. 띄어쓰기 정리
    addr = re.sub(r"\s+", " ", addr)
    
    return addr.strip()


# 전처리된 주소 컬럼
df["정리된주소"] = df["주소"].apply(clean_address)

# Geocoder 설정
geolocator = Nominatim(user_agent="jeonnam-hospital-test")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

# 상위 10개만 테스트
subset = df.head(100).copy()
subset["위도"] = None
subset["경도"] = None
count = 0
# 위경도 출력
for i in range(len(subset)):
    addr = subset.loc[i, "정리된주소"]
    print(f"[{i+1}/10] 주소: {addr}")
    try:
        location = geocode(addr)
        if location:
            lat = location.latitude
            lon = location.longitude
            subset.at[i, "위도"] = lat
            subset.at[i, "경도"] = lon
            print(f"   → 위도: {lat}, 경도: {lon}")
        else:
            print("   → ❌ 위경도 찾을 수 없음")
            count += 1
    except Exception as e:
        print(f"   → ⚠️ 오류: {e}")
print("총 오류 수:", count)


[1/10] 주소: 전라남도 신안군 도초면 서남문로 1515-16
   → 위도: 34.7071294, 경도: 125.939454
[2/10] 주소: 전라남도 영광군 영광읍 신남로 178
   → 위도: 35.2760756, 경도: 126.5015298
[3/10] 주소: 전라남도 담양군 고서면 원등1길 11
   → 위도: 35.2236195, 경도: 126.9749057
[4/10] 주소: 전라남도 담양군 용면 추월산로 369
   → 위도: 35.3650021, 경도: 126.9874564
[5/10] 주소: 전라남도 담양군 봉산면 신학길 8
   → 위도: 35.2744109, 경도: 126.9555414
[6/10] 주소: 전라남도 담양군 월산면 도개길 15-36
   → 위도: 35.3401114, 경도: 126.9561917
[7/10] 주소: 전라남도 완도군 완도읍 개포로62번길 33
   → 위도: 34.3180205, 경도: 126.7476958
[8/10] 주소: 전라남도 고흥군 도양읍 녹동남촌1길 8
   → 위도: 34.5284639, 경도: 127.1383688
[9/10] 주소: 전라남도 고흥군 고흥읍 터미널길 33
   → 위도: 34.6054179, 경도: 127.2801869
[10/10] 주소: 전라남도 고흥군 고흥읍 홍교길 13-1
   → ❌ 위경도 찾을 수 없음
[11/10] 주소: 전라남도 고흥군 고흥읍 원동남계길 26
   → 위도: 34.6092748, 경도: 127.2873108
[12/10] 주소: 전라남도 영광군 영광읍 신남로 168
   → 위도: 35.2760756, 경도: 126.5015298
[13/10] 주소: 가곡동977-6
   → ❌ 위경도 찾을 수 없음
[14/10] 주소: 전라남도 담양군 대전면 덕진옥산길 6
   → ❌ 위경도 찾을 수 없음
[15/10] 주소: 전라남도 여수시 무선로 6
   → 위도: 34.7755283, 경도: 127.6536998
[16/10] 주소: 전라남도 여수시 

In [5]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

# 1) 데이터 불러오기
df = pd.read_csv("전라남도_병원_의원현황.csv", encoding="cp949")

# 2) 주소 컬럼 생성 (도로명 우선, 없으면 지번)
df["주소"] = df["소재지(도로명)"].fillna(df["소재지(지번)"])

# 3) 지오코더 설정
geolocator = Nominatim(user_agent="jeonnam-hospital-test")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)   # 1초 간격 호출

# 4) 테스트용 상위 100개만 복사
subset = df.head(100).copy()
subset["위도"] = None
subset["경도"] = None

error_cnt = 0
total = len(subset)

for i, row in subset.iterrows():
    addr = row["주소"]
    print(f"[{i+1}/{total}] 주소: {addr}")
    try:
        location = geocode(addr)
        if location:
            subset.at[i, "위도"] = location.latitude
            subset.at[i, "경도"] = location.longitude
            print(f"   → 위도: {location.latitude}, 경도: {location.longitude}")
        else:
            print("   → ❌ 위경도 찾을 수 없음")
            error_cnt += 1
    except Exception as e:
        print(f"   → ⚠️ 오류: {e}")
        error_cnt += 1

print("총 오류 수:", error_cnt)

# 필요하면 결과 저장
# subset.to_csv("전남_병원_위경도_100건.csv", index=False, encoding="utf-8-sig")


[1/100] 주소: 전라남도 신안군 도초면 서남문로 1515-16
   → 위도: 34.7071294, 경도: 125.939454
[2/100] 주소: 전라남도 영광군 영광읍 신남로 178, 3층
   → ❌ 위경도 찾을 수 없음
[3/100] 주소: 전라남도 담양군 고서면 원등1길 11
   → 위도: 35.2236195, 경도: 126.9749057
[4/100] 주소: 전라남도 담양군 용면 추월산로 369
   → 위도: 35.3650021, 경도: 126.9874564
[5/100] 주소: 전라남도 담양군 봉산면 신학길 8, 봉산면 보건지소
   → ❌ 위경도 찾을 수 없음
[6/100] 주소: 전라남도 담양군 월산면 도개길 15-36, 월산 보건지소
   → ❌ 위경도 찾을 수 없음
[7/100] 주소: 전라남도 완도군 완도읍 개포로62번길 33, 2층 (금화상사, 아트피아)
   → ❌ 위경도 찾을 수 없음
[8/100] 주소: 전라남도 고흥군 도양읍 녹동남촌1길 8, 소매점 2층
   → ❌ 위경도 찾을 수 없음
[9/100] 주소: 전라남도 고흥군 고흥읍 터미널길 33, 치과병원
   → ❌ 위경도 찾을 수 없음
[10/100] 주소: 전라남도 고흥군 고흥읍 홍교길 13-1
   → ❌ 위경도 찾을 수 없음
[11/100] 주소: 전라남도 고흥군 고흥읍 원동남계길 26
   → 위도: 34.6092748, 경도: 127.2873108
[12/100] 주소: 전라남도 영광군 영광읍 신남로 168, 2층
   → ❌ 위경도 찾을 수 없음
[13/100] 주소: 가곡동977-6(2층)
   → ❌ 위경도 찾을 수 없음
[14/100] 주소: 전라남도 담양군 대전면 덕진옥산길 6, 보건진료소
   → ❌ 위경도 찾을 수 없음
[15/100] 주소: 전라남도 여수시 무선로 6 (학동, 이레타운)
   → ❌ 위경도 찾을 수 없음
[16/100] 주소: 전라남도 여수시 새터로 104, 2층 (신기동)
   → ❌ 위경도 찾을 수 없음
[17/100] 주소

In [ ]:
import requests
import pandas as pd
from geopy.distance import geodesic

REST_API_KEY = "c6167fc9bb70ca9ea487094d849bb1f5"  # 여기에 본인의 REST 키 삽입

headers = {"Authorization": f"KakaoAK {REST_API_KEY}"}
center_lat = 34.9506
center_lon = 127.4872
radius = 5000

category_codes = {
    "음식점": "FD6",
    "편의점": "CS2",
    "공공기관": "PO3"
}

results = []

def search_nearby(category_name, category_code):
    url = "https://dapi.kakao.com/v2/local/search/category.json"
    params = {
        "category_group_code": category_code,
        "x": str(center_lon),   # 반드시 문자열!
        "y": str(center_lat),
        "radius": radius,
        "size": 15
    }

    r = requests.get(url, headers=headers, params=params)
    print(f"{category_name} 상태 코드:", r.status_code)  # 확인용

    if r.status_code == 200:
        for doc in r.json()["documents"]:
            coord = (float(doc["y"]), float(doc["x"]))
            dist = geodesic((center_lat, center_lon), coord).meters
            results.append({
                "이름": doc["place_name"],
                "주소": doc["road_address_name"],
                "위도": doc["y"],
                "경도": doc["x"],
                "거리(m)": int(dist),
                "카테고리": category_name
            })
    else:
        print(f"{category_name} 검색 실패:", r.text)

for name, code in category_codes.items():
    search_nearby(name, code)

df = pd.DataFrame(results)
print(df.head())
df.to_csv("전남_시설_검색결과.csv", index=False, encoding="utf-8-sig")

음식점 상태 코드: 200
편의점 상태 코드: 200
공공기관 상태 코드: 200
         이름                 주소                 위도                  경도  거리(m)  \
0      대원식당  전남 순천시 장천2길 30-29  34.95010189963333  127.49017155220966    276   
1  순천한정식명궁관      전남 순천시 중앙2길 7  34.94982875785895  127.48614014933086    129   
2       화월당    전남 순천시 중앙로 90-1  34.95369370195599  127.48415181634455    441   
3      풍미통닭      전남 순천시 성남뒷길 3  34.94634542197918  127.49094723338463    583   
4      금빈회관      전남 순천시 장명4길 8   34.9515012377406      127.4880582123    127   

  카테고리  
0  음식점  
1  음식점  
2  음식점  
3  음식점  
4  음식점  


In [13]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster

df = pd.read_csv("./데이터/주소위도경도.csv", encoding="utf-8")

# 🔹 위도/경도/이름 컬럼 자동 탐색
lat_col = next((c for c in df.columns if "위도" in c or "lat" in c.lower()), None)
lon_col = next((c for c in df.columns if "경도" in c or "lon" in c.lower()), None)
name_col = next((c for c in df.columns if "폐교명" in c or "name" in c.lower()), None)

if not lat_col or not lon_col:
    raise ValueError("❌ 위도/경도 컬럼이 존재하지 않습니다.")

# 🔹 중심 좌표 계산
df = df.dropna(subset=[lat_col, lon_col])
df[lat_col] = df[lat_col].astype(float)
df[lon_col] = df[lon_col].astype(float)
center = [df[lat_col].mean(), df[lon_col].mean()]

# 🔹 Folium 지도 생성
m = folium.Map(location=center, zoom_start=8, tiles="CartoDB positron")
marker_cluster = MarkerCluster().add_to(m)

# 🔹 마커 추가
for _, row in df.iterrows():
    lat = row[lat_col]
    lon = row[lon_col]
    popup = row[name_col] if name_col else f"{lat}, {lon}"
    folium.Marker([lat, lon], popup=popup).add_to(marker_cluster)

m

In [11]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster

# 🔹 CSV 파일 불러오기 (경로에 맞게 수정)
try:
    df = pd.read_csv("./데이터/주소위도경도.csv", encoding="utf-8")
except UnicodeDecodeError:
    df = pd.read_csv("./데이터/주소위도경도.csv", encoding="cp949")

print(df.columns)      # 모든 컬럼명 확인
print(df.head())       # 몇 줄 미리보기


Index(['NO', '폐교명', '주소', '위도', '경도'], dtype='object')
   NO          폐교명                                                 주소  \
0   1     유달초율도분교장                                 1 전라남도 목포시 율도동 261   
1   2  목포서산초 충무분교장  2 전라남도 목포시 달동 800-8, 800-31, 800-15, 800-19, 8...   
2   3     백초초우두분교장                             3 전라남도 여수시 돌산읍 우두리 170   
3   4  여안초여남중안도분교장                            4 전라남도 여수시 남면 안도리 999-3   
4   5       초도초등학교                            5 전라남도 여수시 삼산면 초도리 산456   

          위도          경도  
0  34.803068  126.322541  
1  34.761947  126.367369  
2  34.714240  127.776903  
3  34.484885  127.805762  
4  34.236548  127.249686  


In [2]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import re

df = pd.read_csv("./데이터/주소위도경도.csv", encoding="utf-8")

print(df.columns)
print(df.head())

Index(['NO', '폐교명', '주소', '위도', '경도', '음식점 수', '편의점 수', '카페 수', '병원 수',
       '주유소/충전소 수', '숙박시설 수'],
      dtype='object')
   NO          폐교명                                                 주소  \
0   1     유달초율도분교장                                   전라남도 목포시 율도동 261   
1   2  목포서산초 충무분교장  전라남도 목포시 달동 800-8, 800-31, 800-15, 800-19, 800...   
2   3     백초초우두분교장                               전라남도 여수시 돌산읍 우두리 170   
3   4  여안초여남중안도분교장                              전라남도 여수시 남면 안도리 999-3   
4   5       초도초등학교                              전라남도 여수시 삼산면 초도리 산456   

          위도          경도  음식점 수  편의점 수  카페 수  병원 수  주유소/충전소 수  숙박시설 수  
0  34.803068  126.322541      1      0     0     0          0      10  
1  34.761947  126.367369    261     21    64     6          8      37  
2  34.714240  127.776903    171     20    41     3          3     140  
3  34.484885  127.805762      7      0     0     0          0      30  
4  34.236548  127.249686      2      0     0     0          0       3  
